# Task07: Multiview Geometry (Development of a 3D Measurement Tool using Structure from Motion)

* **Your Name:** 
* **Your ID:** 

This hands-on demo is designed to impart practical understanding of structure from motion techniques. You are expected to write your own code in the spaces marked **"YOUR CODE HERE"** for each task within this notebook. Please note that some tasks, referred to as "Step", are crucial as they impact subsequent processes. Therefore, ensure that you have executed the preceding code blocks to produce the required outputs.


Last updated: Mar 16, 2024

### Outline
* Step 1: Model generation using VisualSFM
* Step 2: Review output model data exported from VisualSfM 
* Problem 1: Observe the effect of lens (barrel) distortion
* Problem 2: Plot SIFT features and their matches
* Problem 3: Confirming that image features indicate the same world point
* Problem 4: Construct projection batrices (Step 3)
* Problem 5: Exploring the x = PX relationship
* Problem 6: Computing a fundamental matrix
* Problem 7: Drawing and checking epipolar lines
* Problem 8: Point registration and scaling computation (Step 4)
* Problem 9: Compute an unknown distance
* Problm 10: Repeat this process using given differnt image sets 
* Problm 11: Repeat this process using your own image set 



### Step 1: Model generation using VisualSFM

1.  Download and unpack a recent VisualSFM software from the following link 
    http://ccwu.me/vsfm/ 
    Note that you need to download non-cuda version and 64 bit (or cuda
    version if you have a NVIDA graphic card in your machine).
2.  Move the folder of the software in the 'other' folder
3.  Run VisualSFM
4.  Load multi images (File-Oepn Multi images and select all images in img)
5.  Click "Compute missing matches."
6.  Click "Compute 3D reconstruction."
7.  Click "BA (Bundle adjustment)."
8.  Check your constructed model.
9.  Export the model to an NVM file (SfM-Save NView Match) to the folder
    where this code is saved (you need to save it as "result.nvm").

Please take a look at the video if you are unable to get through them.
https://youtu.be/oeQxBM5aQoQ

Be aware that the results from this task **WILL** influence subsequent steps.

### Step 2: Review output model data exported from VisualSfM 

* Please open result.nvm using a notepad (Notepad++ is much easier to read than the notepad. Please google "notepad++") and take look at values and structures. You can find the structure of the NVM file the following link: http://ccwu.me/vsfm/doc.html
* Please thoroughly examine the script provided below. Understanding the creation of `ImgStruct` and `PointInfo` from the nvm file is not required. However, it's crucial that you fully grasp the significance of each element within these two variables.

Be aware that the results from this task **WILL** influence subsequent steps.

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import other.mysfm_fun_set as mysfm

# pip install ipympl 
%matplotlib widget 

# Add path equivalent in Python would be adjusting import paths or sys.path, depending on the context
simgFolder = 'sample'  # sample images (for instruction)
imgFolder = 'img'  # folder of original input images
uimgFolder = 'uimg'  # folder of undistorted images
nvmFileName = os.path.join('result.nvm')  # NVM file name

# Please ensure to open result.nvm with a text editor and review the structure as instructed in the MATLAB comments

ImgStruct, PointInfo = mysfm.read_nvm_file(imgFolder, nvmFileName)
del nvmFileName  # Clear variable

"""
You don't need to look at "Readnvmfile" code in detail, but you should understand the meanings of follwing two dictionaries:

ImgStruct
    ImgStruct['nImg']          : # of original images used for SfM
    ImgStruct['imgSize']       : image size (1x2) in each element
    ImgStruct['imgFileName']   : image file name (string) in each element
    ImgStruct['fMat']          : focal length (double) in each element
    ImgStruct['RMat']          : rotation matrix (3x3) in each element
    ImgStruct['CMat']          : camera center vector (1x3) in each element
    ImgStruct['rDistMat']      : radial distortion (double) in each element

PointInfo
    PointInfo['nPt3D']         : # of 3D points in the world 
    PointInfo['Pt3D']          : location of a 3D point (1x3) in each element
    PointInfo['PtRGB']         : RGB color of a 3D point (1x3) in each element
    PointInfo['nMatchImg']     : # of images of which are used for generating
                                 a 3D point(double) in each element
    PointInfo['matchImgIdx']   : image idexes (nMatchImg x 1) that are used to
                                 generate each 3D point
    PointInfo['matchImgPos']   : position of a feature in each image that
                                 generates a 3D point (nMatchImg x 2) in each
                                 cell
"""

# Note that feature positions stored in PointInfo['matchImgPos'] are the positions on the ORIGINAL images of which lens distortion is not corrected. 

# ===========================================
# View all original images used in this demo
# ===========================================

fileNames = [os.path.join(imgFolder, fname) for fname in ImgStruct['imgFileName']]

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8)) # figsize can be adjusted as needed
axes = axes.flatten() # Flatten the array to easily loop over it

for ax, path in zip(axes, fileNames):
    img = mpimg.imread(path)  # Load the image from path
    ax.imshow(img)  # Display the image on the axes
    ax.axis('off')  # Turn off the axis

for ax in axes[len(fileNames):]:
    ax.axis('off')

plt.tight_layout(), plt.show()

# ==============================
# View point clouds in 3D
# ==============================
xs, ys, zs = zip(*PointInfo['Pt3D'])

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

colors = [f'#{int(r):02x}{int(g):02x}{int(b):02x}' for r, g, b in PointInfo['PtRGB']]
ax.scatter(xs, ys, zs, c=colors, s=3, alpha=1)

ax.set_xlabel('X Axis'), ax.set_ylabel('Y Axis'), ax.set_zlabel('Z Axis')
plt.show()

# ===========================================
# Generate and save undistorted images
# ===========================================

# You don't need to delve into the details of the undistortion process outlined below, but it's essential to execute this code to produce and store the undistorted images. 
# The method for undistortion can differ based on the software used. 
# The undistorted images will be saved in the "uimgFolder", retaining the same filenames as their corresponding original images. 
# NOTE: It's crucial to understand that the pinhole camera geometry is only applicable between the undistorted images (not the original images) and the world points (3D).

if not os.path.exists(uimgFolder):
    os.makedirs(uimgFolder)

for ii in range(len(ImgStruct['imgFileName'])):
    imgName = ImgStruct['imgFileName'][ii]
    if not os.path.exists(os.path.join(uimgFolder, imgName)):
        print(f'Processing undistortion of an image ({ii+1}/{len(ImgStruct["imgFileName"])})')
        
        I = plt.imread(os.path.join(imgFolder, imgName))
        params = {'cx': ImgStruct['imgSize'][ii][0] / 2,
                  'cy': ImgStruct['imgSize'][ii][1] / 2,
                  'r': ImgStruct['rDistMat'][ii] / (ImgStruct['fMat'][ii])**2}
        
        img = mysfm.undistort_image(I, params)
        plt.imsave(os.path.join(uimgFolder, imgName), img)

### Problem 1: Observe the effect of lens (barrel) distortion
Theorectically, all straight lines in the world become staright on the image if they are captured using a perfect pin-hole camera. However, due to radial (barrel) distorion, straight lines are distorted in their view on images. In this task, you will check that a straight line is not staright on an original image but it will be staright on an undistorted image. Please write your own code to check out straight line in both images to demonstrate the effect of lens distortion and its correction. You can make your result similar to the sample plot below:

![](sample/Problem1.png)

Be aware that the results from this task **WILL NOT** influence subsequent steps.

In [ ]:
###########################################################################
######################  YOUR CODE HERE  ###################################
###########################################################################

### Problem 2: Plot SIFT features and their matches

You learned about SIFT features and descriptors in the class. Feature extraction and matching are key processes to enable Structure from Motion (SfM). In this task, convince yourself that the model generated from VisualSFM is constructed based on correct feature matching, which were used to compute the projection matrix for each image as well as its lens distortion parameter. In other words, plot features on any pair of images and draw matching lines between features to verify that they are correctly matched. To complete this task, you need to understand the structure of 'ImgStruct' and 'PointInfo'. You can randomly select any two images. The sample image simply shows the overall match quality. You need to observe one or two feature locations by zooming into those images and include them in your report (a screen capture is fine). Make sure that the points marked on both images correspond to the same points in 3D space. Your outcome should resemble the example plot provided below.:

![](sample/Problem2.png)


**Please think about where we plot the features on either original image or undistorted image**

Be aware that the results from this task **WILL NOT** influence subsequent steps.

In [ ]:
###########################################################################
######################  YOUR CODE HERE  ###################################
###########################################################################

### Problem 3: Confirming that image features indicate the same world point

The objective of this task is to deepen your understanding of the structure and content of the "PointInfo" and "ImgStruct" data structures. From the previous exercise, it was established that features across different images were correctly matched. These matched features are utilized to generate a 3D point, referred to as Pt3D within the dataset. For each 3D point, the specific features and their corresponding images that contributed to its generation are documented. The task involves selecting a 3D point, then plotting all the feature points on their respective images. An important consideration is determining whether to plot these features on the original or the undistorted images.

Here’s how you can approach this:

1. Select a 3D point from the "PointInfo" dataset. This selection can be done either randomly or manually, depending on your preference.
2. Once a 3D point has been selected, identify the corresponding image (feature) points within "PointInfo". These are the points across different images that were used to generate the selected 3D point.
3. Proceed to plot or mark these feature points on their corresponding images. This step is crucial for visually verifying that the plotted feature points across various images indeed represent the same 3D point in the real world.

Your outcome should resemble the example plot provided below.:

![](sample/Problem3.png)

Make sure that the points marked on all images correspond to the same points in 3D space.

It’s essential to understand that this problem does not involve the use of the x = PX equation, which relates world points to their image projections through the camera’s projection matrix. Instead, this task focuses purely on selecting a 3D point and its associated feature points from the "PointInfo", and then visually confirming their correspondence by plotting these features on the images. The computation of the projection matrix for each image is addressed in the subsequent step (Problem 4), highlighting the progression from understanding the data structure to applying it in practical computer vision tasks.

Be aware that the results from this task **WILL NOT** influence subsequent steps.

In [ ]:
###########################################################################
######################  YOUR CODE HERE  ###################################
###########################################################################

###  Problem 4: Construct projection matrices (Step 3)

The next step involves constructing a projective matrix for each image. Refer to the VisualSFM documentation to learn how to construct the $K$ matrix: VisualSFM Documentation (http://ccwu.me/vsfm/doc.html). After determining the $K$ matrices, you will calculate the projective matrix for each image using `ImgStruct['fMat']`, `ImgStruct.['RMat']`, and `ImgStruct.['CMat']`. Remember, these projection matrices are crucial for subsequent steps. You need to store projection matrices in `ImgStruct['PMat']`.

Be aware that the results from this task **WILL** influence subsequent steps. Ensure that the computed projection matrices are stored in `ImgStruct['PMat']` for the subsequent steps. 
Therefore, before advancing to the next phase, verify that the code passes the assert statements.


In [ ]:
###########################################################################
######################  YOUR CODE HERE  ###################################
###########################################################################

<mark> Before proceeding to the next phase, ensure that the code successfully meets the criteria specified by the assert statements.</mark>

In [ ]:
# Assert that the number of projection matrices equals the number of images
assert len(ImgStruct['PMat']) == ImgStruct['nImg'], 'Incorrect size of PMat'
print('You passed the first test!')

# Assert that the size of the first projection matrix is [3, 4]
assert ImgStruct['PMat'][0].shape == (3, 4), 'Incorrect size of a projection matrix'
print('You passed the second test!')

### Problem 5: Exploring the x = PX relationship

In Problem 4, you computed the projection matrix for each image. The equation x = PX illustrates that image points corresponding to any world points can be computed using the projection matrix. It's essential to visually verify that image points projected from a single 3D point (X) accurately represent X. This task differs from Problem 3, which focused on confirming that matched features on original images correspond to the same 3D points (X). However, due to lens distortion, the locations of these features (x) do not exactly match the result of the x=PX equation. Problem 3 aimed to ensure that features used for computing a single 3D point indicate the same 3D point. In contrast, Problem 5 involves confirming that the image points (not the original features) projected from a single 3D point correctly indicate the same world point, provided that the projection matrices for the test images are accurate.

- **Recap of Problem 3**: The task was to verify that "the same" visual points (features) were used to construct a 3D point. These feature locations were originally saved in the NVM file and organized in `PointInfo['matchImgPos']`.

- **Objective of Problem 5**: You are to confirm that image points (not features) projected from a single 3D point accurately represent the same world point when using the correct projection matrices for test images.

You can choose any X point for testing, but it should be visible (captured) in the images. It is recommended to test one of the 3D points listed in PointInfo.Pt3D. Use the projective matrices stored in `ImgStruct['PMat']`, which you computed in Problem 4.

If the distinction between Problems 3 and 5 is unclear, it's crucial to resolve any confusion before proceeding. Understanding the difference is vital for correctly addressing Problem 5. This problem emphasizes the practical application of projection matrices to confirm the accuracy and consistency of projected image points with their corresponding real-world points.

Your outcome should resemble the example plot provided below:

![](sample/Problem5.png)

Be aware that the results from this task **WILL NOT** influence subsequent steps.

### Problem 6: Computing a fundamental matrix

In class, we learned how to compute a fundamental matrix between two images. This process involves computing a matrix, F, using point correspondences. The relationship $x_2' F x_1 = 0$ is valid only under the assumption of pinhole camera geometry. This implies that the locations of features extracted directly from the original images might not yield an accurate $F$, even though these features are used to compute a projection matrix for each image. Instead of relying on these extracted features, we will generate image points by projecting 3D world points onto the images. Then, you'll compute F using the normalized 8-point algorithm based on point correspondences between two images. You'll also need to compute F from the projection matrices of these images using the formula $F=[e]_x P'P^+$.

Here's what you need to do:

1. Choose two images, A and B, either randomly or manually.  
2. Identify all 3D points that can be generated from features visible in both images A and B.  
3. For each 3D point, compute its corresponding image points on A and B using the equation $x=PX$ (here, $x_1$ and $x_2$ are the points on A and B, respectively).  

**Method 1**  

4. Compute $F1$ using the noramlize 8 point algorithm (refer to `two_view_geometry_v1.ipynb` in the tutorial)  
5. Evaluate the average accuracy of your $F1$ using $x_1$ and $x_2$ 

**Method 2**  

4. Compute $F2$ using projection matrices ($F2=[e]_x P' P^+$).
5. Evaluate the average accuracy of your $F2$ using $x_1$ and $x_2$ 

The accuracy of $F1$ and $F2$ can be evaluated using the distance between $l$ and $x_1$, where $l$ is $x_2' * F$. Refer to "distance from a point to a line": https://en.wikipedia.org/wiki/Distance_from_a_point_to_a_line

Your outcome should resemble the example plot provided below.:

![](sample/Problem6.png)

Be aware that the results from this task **WILL NOT** influence subsequent steps.


In [ ]:
###########################################################################
######################  YOUR CODE HERE  ###################################
###########################################################################

### Problem 7: Drawing and checking epipolar lines

This problem focuses on presenting the results obtained from Problem 6, where the main challenge lies in the fact that for a given point $x_1$ in image1, its corresponding point $x_2$ in image2 is unknown due to the absence of the exact location of the corresponding 3D point. However, what is known is that the 3D line equation (ray) where the 3D point is located, and the 2D line equation (epipolar line) in image2 where $x_2$ is positioned. Your task is to draw three epipolar lines in either image1 or image2 to verify if **the corresponding points lie on these lines.**

The concept of epipolar lines is pivotal when we know $X$, $P1$, and $P2$, as it allows us to determine $x_1$ and $x_2$ in images 1 and 2, respectively. These lines are instrumental in computing the fundamental matrix $F$, followed by the computation of P. You have already applied this relationship in Problem 6, where accuracy was assessed by measuring the distance between the epipolar lines and points. This process can be visually replicated as follows:

1. Execute steps 1 to 4 as outlined in Problem 6, ensuring to utilize the data computed therein.
2. Choose three points $x_1$ either randomly or manually.
3. Draw their corresponding epipolar lines on image 2. Here, a question arises: should the image points (x), whether original or undistorted, be plotted on the original image or the undistorted image?
4. Plot $x_2$ on image 2 and verify if $x_2$ is located on the corresponding epipolar lines.

**Ensure that each epipolar line must pass through tht center of the corresponding point.** 

Your outcome should resemble the example plot provided below.:

![](sample/Problem7.png)

In [ ]:
###########################################################################
######################  YOUR CODE HERE  ###################################
###########################################################################

### Problem 8: Point registration and scaling computation (Step 4)

You are required to provide physical dimensional information to your point cloud, meaning that scale information should be added to measure distances between any two 3D points in your point cloud model. The Structure from Motion (SfM) model that you constructed is **up to scale**. To accurately scale your model, you should use a known distance between two points. To determine a scaling factor, you need to register locations of two 3D points and measure the distance between them in your model. The scaling factor is then calculated as the ratio of the physical distance to the model distance you computed.

For the "Bowen wall" example, the distance between bolt holes is 2 feet, and the bolt holes are regularly spaced. This allows you to compute the scaling factor using the space between any two bolt holes. 

![](sample/reference_bowen_wall.jpg)

It is important to note that the two points, whose distance you know, might not be features that you detected earlier. This implies that you need to compute the locations of those 3D points in your model. To do this, you will use a triangulation method to compute a 3D point from 2D points on images, a process we covered in class. **You are expected to implement your own code for triangulation.**

Your task is to write your own triangulation code, following this structure:  
`worldPoints = your_triangulate(matchedPoints1, matchedPoints2, cameraMatrix1, cameraMatrix2).`

Finally, you will compute the scaling and store it in a variable named scalingFactor. You need to pass the provided assert statements to proceed to the next process.

Your outcome should resemble the example plot provided below.:

![](sample/Problem8.png)

Be aware that the results from this task **WILL** influence subsequent steps.

In [ ]:
###########################################################################
######################  YOUR CODE HERE  ###################################
###########################################################################

### Problem 9: Compute an unknown distance
You will measure the distance between any two points in the images. Theoretically, you can take measurements anywhere, but for this example, I focused solely on measuring the distance between bolt holes. I have provided the coordinates for four points on the bolt holes, along with the ground truth measurements below.

![](sample/problem9_result.jpg)

If your `your_triangulate` function and scaling factor are accurate, your measurements should closely match the ground truth. You have to compute the scalingFactor in the previous step. 

Your outcome should resemble the example plot provided below.:

![](sample/Problem9.png)

In [ ]:
###########################################################################
######################  YOUR CODE HERE  ###################################
###########################################################################

### Problm 10: Repeat this process using given differnt image sets  
Two additional datasets, named "Bookshelf" and "Monitor," are provided. The accurate physical measurements (ground truth) are available in the "reference.jpg" file within each dataset. You'll determine a scaling factor using one of these ground-truth measurements. Subsequently, you'll use the remaining measurements for validation purposes, comparing them with the calculations produced by your software. Choose one of the two datasets to work with.

### Problm 11: Repeat this process using your own image set 
Gather a new dataset by capturing images with your camera, which can be the one on your phone. In your scene, take note of several physical distances, and then compare these real-world measurements with the estimates provided by your software.